<a href="https://colab.research.google.com/github/a1xsa/machine_learning/blob/main/bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np              # Одномерные и многомерные массивы (array)
import pandas as pd             # Таблицы и временные ряды (dataframe, series)
import matplotlib.pyplot as plt # Научная графика
import seaborn as sns           # Еще больше красивой графики для визуализации данных
import sklearn                  # Алгоритмы машинного обучения
import os
from sklearn.metrics import accuracy_score
%matplotlib inline

# **Анализ датасета**

# 1. Загружаем данные


Загружаем файл из репозитория Git, изначально данные разделены на тестовую и обучающую выборку, для первоначального анализа и подготовки данных совместим их в одно целое.

In [ ]:
url_train = "https://raw.githubusercontent.com/a1xsa/machine_learning/refs/heads/main/Train.csv"
data_raw_train = pd.read_csv(url_train)
url_test="https://raw.githubusercontent.com/a1xsa/machine_learning/refs/heads/main/Test.csv"
data_raw_test=pd.read_csv(url_test)


In [ ]:
data_raw_train.shape

In [ ]:
data_raw_test.shape

In [ ]:
data_raw_test.head()

In [ ]:
data_all = pd.concat([data_raw_train, data_raw_test], ignore_index=True)
data_all.shape

In [ ]:
data_all.info()

Мы видим что столбцы(признаки) имеют следующие имена:
- `age` - возраст(числовой)
- `job` - работа, категориальный признак ("admin", "unknown","unemployed" "management", "housemaid", "entrepreneur", "student", "blue-collar" "self-employed","retired","technician","services")
- `marital` - семейное положение ("married","divorced","single")
- `education` - образование ("unknown","secondary","primary","tertiary")
- `default` - наличие кредита (бинарный "yes","no")
- `balance` - среднегодовой баланс в евро (числовой)
- `housing` - наличие ипотеки (бинарный "yes","no")
- `loan` - личный кредит (бинарный "yes","no")
- `contact` - тип связи с клиентом ("unknown","telephone","cellular")
- `day` - число последнего контакта с клиентом (числовой)
- `month` - меся последнего контакта с клиентом ("jan", "feb", "mar", …, "nov", "dec")
- `duration` - продолжительность последнего контакта в секундах(числовое)
- `campaign` - количество контактов, выполненных в ходе этой кампании (числовое)
- `pdays` - количество дней, прошедших с момента последнего контакта с клиентом из предыдущей кампании (числовое, -1 - с клианетом ранее не связывались)
- `previous` - количество контактов выполненных до этой кампании
- `poutcome` - результат предыдущей маркетинговой компании ("unknown","other","failure","success")
- `y` - целевой признак, согласие на депозит ("yes", "no")

Требуется предсказать категориальный признак `y` по остальным признакам. Это задача классификации.

In [ ]:
data_all.describe()

Приведем все бинарные и категориальные признаки к типу category.

In [ ]:
data_all['job'].dtype

In [ ]:
data_all['job'] = data_all['job'].astype('category')
data_all['marital'] = data_all['marital'].astype('category')
data_all['education'] = data_all['education'].astype('category')
data_all['default'] = data_all['default'].astype('category')
data_all['housing'] = data_all['housing'].astype('category')
data_all['loan'] = data_all['loan'].astype('category')
data_all['contact'] = data_all['contact'].astype('category')
data_all['month'] = data_all['month'].astype('category')
data_all['poutcome'] = data_all['poutcome'].astype('category')
data_all['y'] = data_all['y'].astype('category')

In [ ]:
data_all['job'].dtype

Выведем информацию о числовых и категориальных признаках.

In [ ]:
data_all.describe()

In [ ]:
data_all.describe(include=['category'])

# 2. Пропущенные значения:

В качестве пропущенных значений будем также принимать поля "unknown".

In [ ]:
data_all.isna().sum()

In [ ]:
value_counts = data_all.apply(lambda col: col.eq("unknown").sum())
print(value_counts)

С пропущенными значениями надо что-то сделать. В качестве обработки неизвестных значений в столбце `poutcome` можно удалить его из рассмотрения в целом. Так же предлагается сделать и со столбцом `contact`. Для признаков `job` и `education` заполним пропущенные значения модой.

In [ ]:
data_all['job'].value_counts()

In [ ]:
data_all['job'] = data_all['job'].replace("unknown", data_all['job'].mode().iloc[0])



In [ ]:
data_all['job'].value_counts()

In [ ]:
data_all['education'].value_counts()

In [ ]:
data_all['education'] = data_all['education'].replace("unknown", data_all['education'].mode().iloc[0])

In [ ]:
data_all['education'].value_counts()

Признаки `contact` и `poutcome` содержат много пропущенных значений, удалим эти столбцы из рассмотрения.

In [ ]:
data_all=data_all.drop(columns=['contact','poutcome'])
value_counts = data_all.apply(lambda col: col.eq("unknown").sum())
print(value_counts)

In [ ]:

data_all.describe(include=['category'])

In [ ]:
data_all.head()

# 3. Работа с выбросами.

In [ ]:
data_all.describe()

In [ ]:
pd.plotting.scatter_matrix(data_all, figsize = (10, 8),alpha=.01)
pass

Выявить выбросы на общей диаграмме затруднительно, используем квантили для решения этой проблемы. Удаляем строки не попавшие в  $99\%$ всех значений.

In [ ]:
data_all['age'].plot(kind='box')
pass

In [ ]:
data_all['balance'].plot(kind='box')
pass

In [ ]:
data_all['balance'].quantile([0.005,.01,.05,.1,.5,.9,.95,.985,.99,.995,.999])

Мы видим что  99%  всех значений признака 'balance' находятся в диапазоне от -800.0 до 32028.0. Удаляем все строки которые находятся за пределами квантилей.

In [ ]:

rows_to_drop = data_all[
    (data_all['balance'] < data_all['balance'].quantile(0.01)) | (data_all['balance'] > data_all['balance'].quantile(0.999))].index
data_all = data_all.drop(rows_to_drop)


In [ ]:
data_all['duration'].plot(kind='box')
pass

In [ ]:
rows_to_drop = data_all[
    (data_all['duration'] < data_all['duration'].quantile(0.01)) | (data_all['duration'] > data_all['duration'].quantile(0.99))].index
data_all = data_all.drop(rows_to_drop)

In [ ]:
data_all['campaign'].plot(kind='box')
pass

In [ ]:
rows_to_drop = data_all[
    (data_all['campaign'] < data_all['campaign'].quantile(0.01)) | (data_all['campaign'] > data_all['campaign'].quantile(0.99))].index
data_all = data_all.drop(rows_to_drop)

In [ ]:
data_all['pdays'].plot(kind='box')
pass

In [ ]:
rows_to_drop = data_all[
    (data_all['pdays'] < data_all['pdays'].quantile(0.01)) | (data_all['pdays'] > data_all['pdays'].quantile(0.99))].index
data_all = data_all.drop(rows_to_drop)

In [ ]:
data_all['previous'].plot(kind='box')
pass

In [ ]:
rows_to_drop = data_all[
    (data_all['previous'] < data_all['previous'].quantile(0.01)) | (data_all['previous'] > data_all['previous'].quantile(0.99))].index
data_all = data_all.drop(rows_to_drop)

In [ ]:
data_all.describe()

In [ ]:
pd.plotting.scatter_matrix(data_all, figsize = (10, 8),alpha=.01)
pass

Теперь диаграмы рассеивания выглядят более "презентабельно". Уже можно рассмотреть некоторые зависимости, к примеру, зависимость возраста от баланса клмента.

# 4. Работа с категориальными признаками:

In [ ]:
data_all.describe(include=['category'])

К категориальным признакам применим метод бинаризации.

In [ ]:
numerical_columns=  [c for c in data_all.columns if data_all[c].dtype.name != 'category']
categorical_columns = [c for c in data_all.columns if data_all[c].dtype.name == 'category']
print(categorical_columns)

In [ ]:
data_describe = data_all.describe(include = ['category'])
binary_columns    = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]
print(binary_columns, nonbinary_columns)


In [ ]:
data_all['y'].value_counts()

In [ ]:
data_all['housing'].value_counts()

Для всех бинарных признаков выше просто заменим значения `yes` и `no` на 1 и 0.

In [ ]:
for column in binary_columns:
  data_all[column] = data_all[column].cat.codes

In [ ]:
data_all.info()

К категориальным небинарным признакам применим векторизацию с помощью `get_dummies`.

In [ ]:
nonbinary_columns

In [ ]:
data_all['job'].unique()

In [ ]:
job_dummies=pd.get_dummies(data_all['job'], dtype=int)

In [ ]:
job_dummies.head(5)

In [ ]:
marital_dummies=pd.get_dummies(data_all['marital'], dtype=int)
marital_dummies.head(5)

In [ ]:
education_dummies=pd.get_dummies(data_all['education'], dtype=int)
education_dummies.head(5)

In [ ]:
month_dummies=pd.get_dummies(data_all['month'], dtype=int)
month_dummies.head(5)

Добавим dummy столбцы к таблице и удалем исходные:

In [ ]:
data_all=pd.concat((data_all,job_dummies),axis=1)
data_all=data_all.drop(['job'], axis=1)

In [ ]:
data_all=pd.concat((data_all,marital_dummies),axis=1)
data_all=data_all.drop(['marital'], axis=1)

In [ ]:
data_all=pd.concat((data_all,education_dummies),axis=1)
data_all=data_all.drop(['education'], axis=1)

In [ ]:
data_all=pd.concat((data_all,month_dummies),axis=1)
data_all=data_all.drop(['month'], axis=1)

In [ ]:
data_all.info()

# 5. Нормализация:

Проведем стандартизацию признаков, чтобы в дальнейшем воспользоваться методом k-ближайших соседей. Стандартизация - линейное преобразование, приводящее все значнеия к нулевому среднему и единичному стандартному отклонению.

In [ ]:
numerical_columns

In [ ]:
data_numerical=data_all[numerical_columns]

In [ ]:
data_numerical=(data_numerical-data_numerical.mean(axis=0))/data_numerical.std(axis=0)

In [ ]:
data_numerical.describe()


# 6. Отбор признаков и разбиение данных:




In [ ]:
data_all=data_all.drop(numerical_columns,axis=1)

In [ ]:
data=pd.concat((data_numerical,data_all),axis=1)

In [ ]:
data.info()

Для решения задачи классификации было принято убрать из рассмотрения признаки `day`, `month`, `pdays`.

In [ ]:
data=data.drop(['day','pdays','apr','aug','dec','feb','jan','jul','jun','mar','may','nov','oct','sep'],axis=1)

In [ ]:
data.info()

Разобьем данные на обучающую и тестовую выборку в пропорции 3:1.(75%- обучающая выборка, 25%- тестовая выборка).

In [ ]:
X=data.drop('y',axis=1)
y=data['y']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)
N_train, _ = X_train.shape
N_test, _ = X_test.shape
N_train, N_test

In [ ]:
y_train

# 7. KNN:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=7, p=1)
model.fit(X_train, y_train)

Проверим качество классификатора.

In [ ]:
y_train_pred=model.predict(X_train)
y_test_pred=model.predict(X_test)

In [ ]:
y_train!=y_train_pred

Ошибки на обучающей и тестовой выборке.

In [ ]:
np.mean(y_train != y_train_pred), np.mean(y_test != y_test_pred)

Построим матрицу рассогласования.

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_test_pred)
classes=[0,1]

In [ ]:
from yellowbrick.classifier import confusion_matrix, class_prediction_error
confusion_matrix(model, X_test, y_test, classes=classes)

In [ ]:
y_test.value_counts()

In [ ]:
class_prediction_error(model, X_test, y_test, classes=classes)

Процент ошибки не велик, однако предсказания по первому классу показывают плохие результаты.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

Подберем параметры оптимального количества соседей.

In [ ]:
# from sklearn.model_selection import GridSearchCV
# nnb = [2, 3, 5, 10, 15, 19]
# knn = KNeighborsClassifier()
# grid = GridSearchCV(knn, param_grid = {'n_neighbors': nnb}, cv=10)
# grid.fit(X_train, y_train)

# best_cv_err = 1 - grid.best_score_
# best_n_neighbors = grid.best_estimator_.n_neighbors
# print(best_cv_err, best_n_neighbors)

In [ ]:
model_19 = KNeighborsClassifier(n_neighbors=19, p=1)
model_19.fit(X_train, y_train)
y_test_pred_19=model_19.predict(X_test)

In [ ]:
confusion_matrix(model_19, X_test, y_test, classes=classes)

In [ ]:
print(classification_report(y_test, y_test_pred_19))

In [ ]:
accuracy_score(y_test, y_test_pred_19)

Как можно заметить, процент ошибки стал меньше, однако определение объектов 1-го класса оставляет желать лучшего.

#8. MLP - Multi Layer Perceptron:

Создадим модель однослойной нейронной сети.

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
model = MLPClassifier(hidden_layer_sizes = (30,), random_state = 42)
model.fit(X_train, y_train)

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
np.mean(y_train != y_train_pred), np.mean(y_test != y_test_pred)

In [ ]:
model_2 = MLPClassifier(hidden_layer_sizes = (1000, 500), random_state = 42)
model_2.fit(X_train, y_train)

In [ ]:
y_train_pred = model_2.predict(X_train)
y_test_pred = model_2.predict(X_test)
np.mean(y_train != y_train_pred), np.mean(y_test != y_test_pred)

Подбор гиперпарамтеров.

Рассмотрим зависимость ошибки от числа нейронов в скрытом слое.

In [ ]:
array_neurons = np.arange(1, 40, 3)
test_err = []
train_err = []
train_acc = []
test_acc = []

for neurons in array_neurons:
    mlp_model = MLPClassifier(hidden_layer_sizes=(neurons), max_iter=1000)
    mlp_model.fit(X_train, y_train)

    y_train_pred = mlp_model.predict(X_train)
    y_test_pred = mlp_model.predict(X_test)

    train_err.append(np.mean(y_train != y_train_pred))
    test_err.append(np.mean(y_test != y_test_pred))

    train_acc.append(accuracy_score(y_train, y_train_pred))
    test_acc.append(accuracy_score(y_test, y_test_pred))

In [ ]:
plt.semilogx(array_neurons, train_err, 'b-o', label = 'train')
plt.semilogx(array_neurons, test_err, 'r-o', label = 'test')
plt.xlim([np.min(array_neurons), np.max(array_neurons)])
plt.title('Dependence Errors', fontsize = 40)
plt.xlabel('Neurons', fontsize = 25)
plt.ylabel('Error', fontsize = 25)
plt.legend(fontsize = 20)

Ошибка на тестовых данных уменьшается с ростом количества нейронов.

Рассмотрим зависимость величины ошибки от параметра `alpha`.

In [ ]:
alpha_arr = np.logspace(-4, 5, 10)
test_err = []
train_err = []
for alpha in alpha_arr:
    mlp_model = MLPClassifier(hidden_layer_sizes = (30,), random_state = 42, alpha = alpha)
    mlp_model.fit(X_train, y_train)

    y_train_pred = mlp_model.predict(X_train)
    y_test_pred = mlp_model.predict(X_test)
    train_err.append(np.mean(y_train != y_train_pred))
    test_err.append(np.mean(y_test != y_test_pred))

In [ ]:
plt.semilogx(alpha_arr, train_err, 'r-o', label = 'train')
plt.semilogx(alpha_arr, test_err, 'b-o', label = 'test')
plt.xlim([np.max(alpha_arr), np.min(alpha_arr)])
plt.title('Error vs. alpha')
plt.xlabel('alpha')
plt.ylabel('error')
plt.legend()
pass